In [4]:
#import gensim
from gensim import corpora, models, similarities
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
def call_mysql():
    # pull in data from the reviews table
    con = False
    rows = np.nan
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT id,business_id,stars,comment FROM review;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
#             for row in rows:
#                 print(row)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [6]:
rows = call_mysql()

# UNDERSTAND THE OUTPUT OF THE MYSQL
# print(np.shape(rows))#(3616, 4)
# print(type(rows))#tuple
# print(type(rows[:][0]))# still tuple
# print(type(rows[3][3]))# int/str or whatever per the column

#print(rows[0][3])#just the review for the zero-indexed row [0]
# print(rows[1][3])#the review for the one-indexed row [1]

# confusing - not printing all rows of the first column
#print(rows[:][0])#all four columns of the data-row

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

print(documents[0])# prints one review

# for doc in range(0,ndocs):
#     [review for review in rows[:][0]]# This prints the review from the id in the second []
#     print review#type str

# print(row[3])#just the review
# print(row[:])# a row of data
# print(review)# one review


Adding some more information. Another great thing about him is that he was willing to experiment with me for my fibromyalgia. I brought in a book by a specialist and he was willing to try the protocol. Many doctors would not do that. He won't do anything crazy but will consider something reasonable. He also made himself available when I was having an emergency severe toothearache. He was at an offsite seminar, but his receptionist asked him to call me and he did, prescribed something and I think he even checked in with me later on or I called back. Either way, I got the treatment I needed.For those who think it takes a while to get ahold of him or his receptionistplease keep in mind it is one doctor and one receptionist. As you can see from all of the 5 star reviews, he spends time with his patients. Be a little patient yourself and you will be soon benefit from his great treatment.


In [7]:
def unify_time(word):
    #Assign all possible time values ('9:45','12:05',etc) to one word 'oclock'
    match = re.search(r'\d*'+':'+r'\d*',word)
    if match:
        #print(match.group())
        word = 'oclock'
    return word


    # This also works, better if you want to catch the error and do something, but I don't
    # try: 
    #     match = re.search(r'\d*'+':'+r'\d*',s, word)
    #     test = match.group() 
    #     word = 'oclock'
    # except AttributeError as e: 
    #     print('error in unify_time')

In [8]:
def period_split(word):
    # Split words with a period in the middle - not yet the ideal implementation b/c it returns two words as one
    match = re.search(r'\w*'+'.'+r'\w*',word)
    if match:
        word = word.replace('.',' ')
    return word

In [9]:
def canonicalize(word):    
    bads = ['dr.',"n't","'ve","'re",'wo',"'m"]
    goods = ['dr','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [10]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",'I','me','mine','my','you','your',
            'yours','it','its','they','their','theirs','we','us','our','ours']
    few = ['male','male','male','female','female','female','firstper','firstper',
           'firstper','firstper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [29]:
def clean(word):
    # wrapper function - performs a bunch of functions on a given word
    word = unify_time(word)
    word = unify_pronouns(word)
    word = canonicalize(word)
    word = period_split(word)    
    return word

In [30]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [87]:
examples = ['tamalpais-pediatrics-novato','name-name-name']
final = []
names = []

def split_append(yelp_id):
    yelp_id = yelp_id.split('-')
    #print(yelp_id)
    return yelp_id

# for ex in examples:
#     print(ex)
    
#names.append(split_append(examples[1]))

names = [[names.append[e][ex] for e in ex] 

# for ex in examples:
#     names = names.append(split_append(ex))
#     print(names)
#     for name in names:
#         print(name)
#         final = final.append(name)
# print(final)
#print(names)


None


TypeError: 'NoneType' object is not iterable

In [71]:
# NLTK:  remove common words and tokenize

# Explore this
#from nltk.corpora import stopwords
#[print(w) for w in stopwords.words()]

common_words = 'for a as was "was" is of the and this that "that" sf to in if "," , ,, \'s by on the to . .. ...  `` ? & * _ - / "\" 3 2 5'
common_names = 'kong lisa chueng cheung dolev laroque caluag khoo usman kim jacoby mccaw simons wong eshima'
nums = numwords()
common_words = common_words.join([' '+nums+' '+common_names])

stoplist = set(common_words.split())

# Investigate this to remove punctuation
# from nltk.tokenize import RegexpTokenizer
# RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
# texts = [[word for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
#         for document in documents]#review in reviews 
texts = [[clean(word) for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
        for document in documents]#review in reviews 

print(texts[:2])

[['adding', 'some', 'more', 'information', ' ', 'another', 'great', 'thing', 'about', 'male', 'is', 'that', 'male', 'was', 'willing', 'to', 'experiment', 'with', 'firstper', 'for', 'firstper', 'fibromyalgia', ' ', 'i', 'brought', 'in', 'a', 'book', 'by', 'a', 'specialist', 'and', 'male', 'was', 'willing', 'to', 'try', 'the', 'protocol', ' ', 'many', 'doctors', 'would', 'not', 'do', 'that', ' ', 'male', 'will', 'not', 'do', 'anything', 'crazy', 'but', 'will', 'consider', 'something', 'reasonable', ' ', 'male', 'also', 'made', 'himself', 'available', 'when', 'i', 'was', 'having', 'an', 'emergency', 'severe', 'toothearache', ' ', 'male', 'was', 'at', 'an', 'offsite', 'seminar', ',', 'but', 'male', 'receptionist', 'asked', 'male', 'to', 'call', 'firstper', 'and', 'male', 'did', ',', 'prescribed', 'something', 'and', 'i', 'think', 'male', 'even', 'checked', 'in', 'with', 'firstper', 'later', 'on', 'or', 'i', 'called', 'back', ' ', 'either', 'way', ',', 'i', 'got', 'the', 'treatment', 'i', '

In [72]:
# BAG OF WORDS
dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
#dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/doctor.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(20108 unique tokens: [u'risk  etc', u'200 00', u'radiologists', u'buccal', u'yellow']...)


In [11]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('/tmp/doctor.mm', corpus) # store to disk, for later use

#print(corpus)
# looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

In [1]:
# retrain model with more data
# sort documents based on topic assignments
# e.g., look at the top five documents assigned to this topic to get a sense what it's about
# you train model on many doctors' reviews, but when you do the prediction, you can project
# back onto a single doctor
# each review might contain a weightings of topics - 70%(T1)+30%(T2)
# give each new doctor a star rating per topic - that's the readout - personalized report

In [ ]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# I think you can't give it the tfidf_corpus - seems not to work/not sure why though
# lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=5, alpha=auto)
# doc_lda = lda[corpus_tfidf]

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, update_every=0, passes=10, iterations=100)

In [82]:
# 2) Infer topic distributions on new, unseen documents:
#doc_lda = lda[mmcorpus]
doc_lda = lda[corpus]
print(lda.inference(corpus))

# 3) Update model with new documents:
#lda.update(new_training_corpus)
#lda.print_topics(5)

# row = [[d[1] for d in doc] for doc in corpus_lsi]
# df = pd.DataFrame(row).sort(0,axis=0,ascending=False)#0 sorts on the first column

(array([[  1.00010082e-01,   1.00026562e-01,   1.00015552e-01, ...,
          1.00027290e-01,   1.00026297e-01,   1.00030079e-01],
       [  1.00009776e-01,   1.00024731e-01,   1.00015130e-01, ...,
          1.00027703e-01,   2.77113822e+01,   3.81945556e+02],
       [  1.00015560e-01,   1.00032430e-01,   1.00019380e-01, ...,
          1.00027670e-01,   1.00029240e-01,   1.10099797e+02],
       ..., 
       [  1.00016950e-01,   1.82096042e+01,   1.00011275e-01, ...,
          1.00025958e-01,   1.00025516e-01,   1.00025544e-01],
       [  1.00011773e-01,   4.59893128e+01,   1.00014998e-01, ...,
          1.00029694e-01,   1.00027997e-01,   1.00027944e-01],
       [  1.00011111e-01,   1.00028908e-01,   1.00014624e-01, ...,
          4.02927939e+00,   1.00029649e-01,   1.00027219e-01]]), None)


In [68]:
print(lda.top_topics(corpus, num_words=5)[:][0])
lda.top_topics(corpus, num_words=5)#shape (10,2) for num_words=5
#lda.print_topics(10)

#row = [[d[1] for d in doc] for doc in doc_lda]
#df = pd.DataFrame(row).sort(0,axis=0,ascending=False)#0 sorts on the first column

# Old comment from gensim_tut1
# As expected, the first five documents are more strongly related to the second topic 
# while the remaining four documents to the first topic:

# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
# for doc in corpus_lsi: 
#     print(doc)

([(0.044426429282918886, u'female'), (0.024594040190564603, u'and'), (0.020746306400343757, u','), (0.019574843276996073, u' '), (0.014199064001544255, u'i')], -0.86004217862364207)


[([(0.044426429282918886, u'female'),
   (0.024594040190564603, u'and'),
   (0.020746306400343757, u','),
   (0.019574843276996073, u' '),
   (0.014199064001544255, u'i')],
  -0.86004217862364207),
 ([(0.045786505507638152, u','),
   (0.037335262883056261, u' '),
   (0.030974352889445844, u'and'),
   (0.0302371329613676, u'to'),
   (0.028789394479197681, u'the')],
  -0.97312414397536162),
 ([(0.035792451040030329, u','),
   (0.029614682929964782, u'to'),
   (0.028963535614022932, u'the'),
   (0.025669236589629635, u'i'),
   (0.025408723582341104, u' ')],
  -1.1007921390186388),
 ([(0.050442486506677556, u'i'),
   (0.045048420342704669, u' '),
   (0.036975271847549764, u'and'),
   (0.036736057798340091, u'firstper'),
   (0.028654083352738086, u'to')],
  -1.1478667613391202),
 ([(0.047161136866373292, u' '),
   (0.035680371493364352, u'i'),
   (0.0289893511445613, u','),
   (0.025369317657303905, u'the'),
   (0.023350364611845566, u'and')],
  -1.2215366066574709),
 ([(0.04142745344264105

In [46]:
# from nltk.data import find
# from nltk.parse import bllip
# #from bllipparser import RerankingParser, tokenize

# sentence = 'I really liked this doctor. We really got along well.  She was really easy to talk to.'
# new = nltk.word_tokenize(sentence)
# print(new)

# #nltk.parse.bllip.BllipParser()
# model_dir = find('models/bllip_wsj_no_aux').path
# #bllip = bllip.BllipParser.from_unified_model_dir(model_dir)
# top_parse = bllip.parse_one(sentence)#not working

In [87]:
# Create a new corpus & apply LDA model to the new corpus
# https://radimrehurek.com/gensim/models/ldamodel.html#id2
# https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation

new_docs = [u"""We love Tamalpais Pediatrics and our children's doctor, John Lee. Like other reviewers have found, with their doctors, 
           Dr. Lee is patient, listens and addresses concerns we have about our childrens health.  I don't know if all pediatricians will come to the hospital to meet the new baby, but Dr. Lee has come to see both our daughters after they were born...
           We've seen Dr. Branco when Dr. Lee was unavailable and were relieved to see he and Dr. Lee are very similar in bedside manners.  
        Crystal Cox, the PA, is also a dream.  Shes warm and very easygoing with our young children.  I've no problems having her see the kids when one of the MD s are not available.
        I'm sorry to see Deana S. did not have a good experience with the receptionist, but I have to say that I've never had any problems with any of the staff.  They are all professional and very caring with the children.  
        5 Stars!  Thumbs up!"""]
# QUICK FORMAT: remove extra spaces and new lines
new_doc = [" ".join(element.replace("\n","").split()) for element in new_doc]
print(np.shape(new_doc))
#print(new_doc)

# I'll want to make this a function that I call later on
# NLTK code
# remove common words and tokenize
common_words2 = 'for a of the and to in is \'s by on the to , . `` ? _ - / "\"'
stoplist2 = set(common_words2.split())

#texts: A list of lists; text: list of tokenized words from 1 doc/review
texts2 = [[word2 for word2 in nltk.word_tokenize(doc.lower()) if word2 not in stoplist2]
        for doc in new_docs]#review in reviews 
#print(texts2)

# bag of words
new_dictionary = corpora.Dictionary(texts2)# create dictionary object from cleaned texts
new_dictionary.save('/tmp/tamalpais_pediatrics.dict') # store the dictionary, for future reference
print(new_dictionary)
#print(new_dictionary.token2id)

# form new corpus
new_corpus = [new_dictionary.doc2bow(text) for text in texts2]
# A trained model can used be to transform new, unseen documents 
# (plain bag-of-words count vectors) into LDA topic distributions:

# get topic probability distribution for a new review: TransformedCorpus' object
new_doc_lda = lda[new_corpus]
#new_doc_lda.print_topics()#doesn't work
print(new_doc_lda)#This just prints the object: <gensim.interfaces.TransformedCorpus object at 0x10d3ccf10>
# throws away sstats, the second part of the returned tuple
gammas,_ = lda.inference(new_corpus)# first element is gammas matrix: topics by documents - gamma (parameters controlling the topic weights) for each document in the chunk
# Is normalization needed
norm_gammas = gammas/np.sum(gammas,axis=1)
print(norm_gammas)

# KEY QUESTIONS/To DO
# *** How do I see the results applied to my new document???
# *** I still don't know what to graph
# *** How do I get an example review that is highly "representative of a given topic?"
# I still don't know the best way to reduce the topic dimensonality space
# I don't understand the LDA results, and why stop words are not fully suppressed
# Need to remove the access key from my git file for the YELP data API program
# Need to pass an argument from webpage to flask, manipulate it and pass it back to output page
# Add insight to linked in
# Work on resume
# Revise the Yelp API program and split into two programs - 1) get businesses, 2) get reviews

(1,)
Dictionary(101 unique tokens: [u'all', u'reviewers', u'bedside', u'thumbs', u'tamalpais']...)
[[  6.80334366e-04   6.80435150e-04   6.80367168e-04   7.22134614e-02
    6.80463070e-04   9.22343258e-01   6.80272172e-04   6.80459366e-04
    6.80470621e-04   6.80478184e-04]]


In [86]:
# Visualization

vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)
